<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -q
!wandb login

     |████████████████████████████████| 1.9 MB 8.3 MB/s 
     |████████████████████████████████| 166 kB 64.5 MB/s 
     |████████████████████████████████| 182 kB 73.1 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 166 kB 71.8 MB/s 
     |████████████████████████████████| 162 kB 74.6 MB/s 
     |████████████████████████████████| 162 kB 70.7 MB/s 
     |████████████████████████████████| 158 kB 72.6 MB/s 
     |████████████████████████████████| 157 kB 77.8 MB/s 
     |████████████████████████████████| 157 kB 65.3 MB/s 
     |████████████████████████████████| 157 kB 77.2 MB/s 
     |████████████████████████████████| 157 kB 77.7 MB/s 
     |████████████████████████████████| 157 kB 77.6 MB/s 
     |████████████████████████████████| 157 kB 76.9 MB/s 
     |████████████████████████████████| 157 kB 74.1 MB/s 
     |████████████████████████████████| 156 kB 69.1 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: ht

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
vgg16 = VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

58889256/58889256 [==============================] - 0s 0us/step


In [6]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

In [7]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.


In [8]:
class_names = train_ds.class_names
test_class_names = test_ds.class_names
class_names == test_class_names
print(class_names)
num_classes = len(class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [9]:
model_vgg16 = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  vgg16,
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
vgg16.trainable=False
model_vgg16.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 32)                802848    
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 15,517,701
Trainable params: 803,013
Non-t

In [10]:
model_vgg16.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [11]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "vgg16",
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [12]:
%%time
epochs=40
history = model_vgg16.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 0.9787 - accuracy: 0.6278

wandb: Adding directory to artifact (/content/wandb/run-20221104_123955-3e2zbmb8/files/model-best)... Done. 0.8s


22/22 [==============================] - 106s 3s/step - loss: 0.9787 - accuracy: 0.6278 - val_loss: 0.5525 - val_accuracy: 0.8114
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 0.5240 - accuracy: 0.7727

wandb: Adding directory to artifact (/content/wandb/run-20221104_123955-3e2zbmb8/files/model-best)... Done. 0.7s


...

22/22 [==============================] - 12s 512ms/step - loss: 0.0806 - accuracy: 0.9673 - val_loss: 0.0891 - val_accuracy: 0.9657
Epoch 21/40
22/22 [==============================] - 8s 344ms/step - loss: 0.0660 - accuracy: 0.9787 - val_loss: 0.0930 - val_accuracy: 0.9600
Epoch 22/40
22/22 [==============================] - 8s 341ms/step - loss: 0.0878 - accuracy: 0.9673 - val_loss: 0.0897 - val_accuracy: 0.9657
Epoch 23/40
22/22 [==============================] - 8s 341ms/step - loss: 0.0787 - accuracy: 0.9744 - val_loss: 0.1126 - val_accuracy: 0.9600
Epoch 24/40
22/22 [==============================] - ETA: 0s - loss: 0.0900 - accuracy: 0.9616

wandb: Adding directory to artifact (/content/wandb/run-20221104_123955-3e2zbmb8/files/model-best)... Done. 0.7s


22/22 [==============================] - 12s 522ms/step - loss: 0.0900 - accuracy: 0.9616 - val_loss: 0.0889 - val_accuracy: 0.9600
Epoch 25/40
22/22 [==============================] - 8s 344ms/step - loss: 0.0662 - accuracy: 0.9730 - val_loss: 0.1212 - val_accuracy: 0.9486
Epoch 26/40
22/22 [==============================] - 8s 345ms/step - loss: 0.0898 - accuracy: 0.9616 - val_loss: 0.1058 - val_accuracy: 0.9600
Epoch 27/40
22/22 [==============================] - 8s 342ms/step - loss: 0.0830 - accuracy: 0.9659 - val_loss: 0.1089 - val_accuracy: 0.9486
Epoch 28/40
22/22 [==============================] - 8s 343ms/step - loss: 0.0626 - accuracy: 0.9744 - val_loss: 0.1175 - val_accuracy: 0.9429
Epoch 29/40
22/22 [==============================] - 8s 348ms/step - loss: 0.0815 - accuracy: 0.9631 - val_loss: 0.1327 - val_accuracy: 0.9543
Epoch 30/40
22/22 [==============================] - 8s 350ms/step - loss: 0.0773 - accuracy: 0.9673 - val_loss: 0.0932 - val_accuracy: 0.9600
Epoch 31/4

wandb: Adding directory to artifact (/content/wandb/run-20221104_123955-3e2zbmb8/files/model-best)... Done. 0.7s


22/22 [==============================] - 12s 515ms/step - loss: 0.0631 - accuracy: 0.9702 - val_loss: 0.0874 - val_accuracy: 0.9600
CPU times: user 2min 7s, sys: 33 s, total: 2min 40s
Wall time: 10min 8s


In [13]:
wandb.finish()

accuracy,▁▄▅▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▄▆▆▇▇▇▇▇▇▇▇█▇▇█▇▇█████▇█▇▇▇███▇██▇▇▇██
val_loss,█▆▅▄▄▃▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▂▁▁▁▂▁▁▁▂▁▁▂▁▂▂▁
accuracy,0.97017
best_epoch,39
best_val_loss,0.08738
epoch,39
loss,0.06311
val_accuracy,0.96


In [14]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
